In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import saraRC1 as sara

import scipy.stats as sc

seg_dim = 8

In [2]:
T = 0.001

def index_to_coordinates(index, seg_dim, im_size):
    '''
    Given an index and a shape, this function returns the corresponding coordinates.
    '''

    x1 = int((index % seg_dim) * (im_size[1] / seg_dim))
    y1 = int((index // seg_dim) * (im_size[0] / seg_dim))

    x2 = int(x1 + (im_size[1] / seg_dim))
    y2 = int(y1 + (im_size[0] / seg_dim))
    
    return (x1, y1, x2, y2)

def mae(pred, gt):
    return np.mean(np.abs(pred - gt))

results = {}

In [4]:
ranges = list(zip(np.arange(1, 7000, 100), np.arange(100, 7100, 100)))
ranges = [list(t) for t in ranges]
ranges[0][0] = 0

img_path = './Siris Dataset/ASSR/images/train/'

experiment = 1

for experiment in range(1, 10):
    print('EXPERIMENT:', experiment)
    range_ = ranges[experiment - 1]
    imgs = {}
    i = 0

    ### IMAGES
    for root, dirs, files in os.walk(img_path):
        for file in files[range_[0]:range_[1]]:
            file_name = file.split('.')[0]
            imgs[file_name] = cv2.cvtColor(cv2.imread(os.path.join(root, file)), cv2.COLOR_BGR2RGB)
            i += 1


    ### MASKS
    mask_path = './Siris Dataset/ASSR/gt/train/'
    # masks = {}
    gt_masks = {}
    gt_ranks = {}

    for file_name in imgs:
        gt_ranks[file_name] = {}
        
        file = file_name + '.png'
        mask = cv2.cvtColor(cv2.imread(os.path.join(mask_path, file)), cv2.COLOR_BGR2GRAY)
        # masks[file_name] = cv2.cvtColor(cv2.imread(os.path.join(mask_path, file)), cv2.COLOR_BGR2GRAY)
        # Separate the mask based on colour
        # masks[file_name] = {}
        gt_masks[file_name] = {}

        # Detect different colours in mask
        # Create histogram
        hist = cv2.calcHist([mask], [0], None, [256], [0, 256])

        # Show non-zero values and extract intensity values at that freq
        non_zero = np.nonzero(hist)
        x = non_zero[0][1:]

        # Separate mask into regions which match the intensity values in x
        for i, intensity in enumerate(reversed(x)):
            # masks[file_name][i] = np.where(mask == intensity, 1, 0)
            gt_masks[file_name][i] = np.where(mask == intensity, 1, 0)
            
            # Calculate ranks based on highest intensity
            gt_ranks[file_name][i] = i + 1

    masks = gt_ranks # To be replaced with MASK R-CNN

    ### GENERATE SALIENCY MAPS, HEATMAPS, RANKINGS
    saliency_maps = {}
    sara_heatmaps = {}
    sara_lists = {}

    from time import time

    plt.rcParams['figure.figsize'] = [20, 10]
    plt.figure()
    plt.tight_layout()

    # generators = ['itti', 'deepgaze', 'fpn', 'emlnet']
    generators = ['deepgaze']

    g = 1

    # DeepGaze
    import deepgaze_pytorch
    DEVICE = 'cuda'
    model = deepgaze_pytorch.DeepGazeIIE(pretrained=True).to(DEVICE)

    for i, im in enumerate(imgs):
        start = time()

        if im not in saliency_maps:
            saliency_maps[im] = {}
            sara_heatmaps[im] = {}
            sara_lists[im] = {}

        for generator in generators:
            percent = round(g/(len(imgs)*len(generators))*100, 2)
            
            print(f'--> Processing image {i+1} of {len(imgs)} ({generator}) [{percent}%]')

            saliency_maps[im][generator] = sara.return_saliency(imgs[im].copy(), generator=generator, deepgaze_model=model, emlnet_models=emlnet_models)
            sara.reset()

            sara_heatmaps[im][generator], sara_lists[im][generator] = sara.return_sara(imgs[im].copy(), seg_dim, saliency_map=saliency_maps[im][generator])
            sara.reset()

            g += 1
        
        print(f'Time for image {i + 1}: {time() - start} seconds')

    ### MASK RANKING
    # For each segment, check which mask falls under that segment using MRn = rank(Gi); (Gi interesect Mn) > T
    mask_segments = {}

    for sara_list in sara_lists:
        for segment in sara_lists[sara_list]['deepgaze']:
            if sara_list not in mask_segments:
                mask_segments[sara_list] = {}

            # Convert index to coordinates, extract segment from heatmap
            shape = sara_heatmaps[sara_list]['deepgaze'].shape[0:2]
            x1, y1, x2, y2 = index_to_coordinates(segment[5], seg_dim, shape)
            # print(x1, y1, x2, y2)

            for m in masks[sara_list]:
                if m not in mask_segments[sara_list]:
                    mask_segments[sara_list][m] = []

                # Extract mask from masks
                mask = masks[sara_list][m][y1:y2, x1:x2]

                # Calculate intersection over union
                intersection = np.sum(mask > 0)
                union = np.sum(mask > 0) + np.sum(mask == 0)

                iou = intersection / union

                # print('Segment: ', segment[5], 'Mask: ', m, 'IoU: ', iou)

                if iou > T:
                    # index, rank, saliency
                    # print(segment)
                    mask_segments[sara_list][m].append((segment[5], segment[0], segment[1]))
                    # print(mask_segments)

    # For each mask, find the segment with the lowest rank
    mask_segments_min = {}

    for sara_list in mask_segments:
        for m in mask_segments[sara_list]:
            # mask_segments_min[sara_list][m] = min(mask_segments[sara_list][m], key=lambda x: x[1])[0]
            if sara_list not in mask_segments_min:
                mask_segments_min[sara_list] = {}
            
            mask_segments_min[sara_list][m] = min(mask_segments[sara_list][m], key=lambda x: x[1])

    mask_segments_min

    mask_ranks = {}

    for sara_list in mask_segments_min:
        mask_ranks[sara_list] = {}
        # Extract the ranks and sort them by the third value in each tuple
        sorted_ranks = sorted(mask_segments_min[sara_list].items(), key=lambda x: x[1][1])

        for i in range(len(sorted_ranks)):
            mask_ranks[sara_list][sorted_ranks[i][0]] = i + 1

        # Sort mask_ranks[sara_list] by object
        mask_ranks[sara_list] = {k: v for k, v in sorted(mask_ranks[sara_list].items(), key=lambda item: item[0])}


    ### SOR
    all_spr = 0

    for sara_list in mask_ranks:
        print(f'PR: {list(mask_ranks[sara_list].values())}\nGT: {list(gt_ranks[sara_list].values())}')
        try:
            spr = sc.spearmanr(list(mask_ranks[sara_list].values()), list(gt_ranks[sara_list].values()))
            all_spr += spr.correlation
        except:
            pass
        print()

    print(f'Average SPR: {all_spr / len(mask_ranks)}')

    all_mae = 0

    for sara_list in masks:
        for mask in masks[sara_list]:
            # print(masks[sara_list][mask] - gt_masks[sara_list][mask])
            pred = masks[sara_list][mask]
            pred = np.array(pred, dtype=np.uint8)
            pred = cv2.threshold(pred, 0.5, 1, cv2.THRESH_BINARY)[1]

            gt = gt_masks[sara_list][mask]
            gt = np.array(gt, dtype=np.uint8)
            gt = cv2.threshold(gt, 0.5, 1, cv2.THRESH_BINARY)[1]

            temp_mae = mae(pred, gt)
            # plt.figure()
            # plt.subplot(131)
            # plt.imshow(pred)
            # plt.subplot(132)
            # plt.imshow(gt)
            # plt.subplot(133)
            # plt.imshow(pred - gt)
            all_mae += temp_mae
            # print()
            break

    print(f'Average /mae: {all_mae / len(masks)}')

    results[experiment] = {}
    results[experiment]['SOR'] = all_spr / len(mask_ranks)
    results[experiment]['MAE'] = all_mae / len(masks)
    

EXPERIMENT: 1
Loaded pretrained weights for efficientnet-b5


Using cache found in /home/matthewkenely/.cache/torch/hub/pytorch_vision_v0.6.0
Using cache found in /home/matthewkenely/.cache/torch/hub/pytorch_vision_v0.6.0


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 5.78 GiB total capacity; 399.23 MiB already allocated; 39.19 MiB free; 402.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

<Figure size 1440x720 with 0 Axes>